In [1]:
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException



import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome("C:\\Users\\Umesh Gupta\\chromedriver_win32\\chromedriver.exe")

In [4]:
driver.get("https://www.flipkart.com")

In [5]:
search_box = driver.find_element_by_xpath('//input[@class="_3704LK"]')

In [6]:
search_btn = driver.find_element_by_xpath('//div[@class="col-12-12 _2oO9oE"]//button')

In [7]:
#parameter for which we need to fetch data
key = ["Smart watches","Phones","Laptop","Professional Camera","Headphones","Printers","Monitors","Router","Home Theatre"]

In [10]:
#fetching urls for scraping data
urls = []
for i in key:
    time.sleep(1)
    search_box.clear()
    search_box.send_keys(i)
    search_btn.click()
    time.sleep(3)
    #fetch data for only two pages
    for pages in range(0,2):
        time.sleep(1)
        for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK" or @class="s1Q9rs"]'):
            urls.append(i.get_attribute('href'))
        button=driver.find_elements_by_xpath("//*[contains(@class, '_1LKTO3')]")
        time.sleep(1)
        for k in button:
            if k.text=="NEXT":
                k.click()
                

In [13]:
#url fetched
url=urls[0:500]
len(url)

500

In [14]:
#fetching review and rating
Rating = [] # Creating list to store data
full_r = []
#call every url and click on all review button to fetch review
for i in url:
    driver.get(i)
    time.sleep(3)
    try:
        all_review =driver.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']//span")
        all_review.click()
        time.sleep(2)
    except NoSuchElementException:
        continue
    #fetching review of 10 pages
    for i in range(0,9):
        try:
            for i in driver.find_elements_by_xpath("//div[@class='_3LWZlK _1BLPMq' or @class='_3LWZlK _32lA32 _1BLPMq' or @class='_3LWZlK _1rdVr6 _1BLPMq']"):
                if i.text is None:
                    Rating.append("---")
                else:
                    Rating.append(i.text.split()[0])
                
        except NoSuchElementException or StaleElementReferenceException:
            Rating.append("---")  

        try:
            for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']/div/div"):
                full_r.append(i.text)
        except:
            None
        
    
        try:
            button=driver.find_element_by_xpath("//*[contains(text(), 'Next')]")
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
        
       
        except:
             break
    

In [15]:
#verifying len of review and rating fetched
print(len(Rating),len(full_r))

25544 25544


In [17]:
#creating dataframe
review=pd.DataFrame({"Review":full_r,
             "Rating":Rating})

In [19]:
review.head()

,Review,Rating
0,Bad camera no use average watch,3
1,It's amazing watch more features is available\...,3
2,Love,5
3,Ghatiya watch ghatiya company dont buy this watch,1
4,Battery damage,5


In [20]:
#saving to csv file
review.to_csv("review.csv")